In [17]:
import pandas as pd 

train = pd.read_csv('data/train.csv')
train

,Unnamed: 0,filename,chip_id,satellite,month,corresponding_agbm
0,0,0003d2eb_S1_00.tif,0003d2eb,S1,September,0003d2eb_agbm.tif
1,1,0003d2eb_S1_01.tif,0003d2eb,S1,October,0003d2eb_agbm.tif
2,2,0003d2eb_S1_02.tif,0003d2eb,S1,November,0003d2eb_agbm.tif
3,3,0003d2eb_S1_03.tif,0003d2eb,S1,December,0003d2eb_agbm.tif
4,4,0003d2eb_S1_04.tif,0003d2eb,S1,January,0003d2eb_agbm.tif
...,...,...,...,...,...,...
189073,252398,fff05995_S2_07.tif,fff05995,S2,April,fff05995_agbm.tif
189074,252399,fff05995_S2_08.tif,fff05995,S2,May,fff05995_agbm.tif
189075,252400,fff05995_S2_09.tif,fff05995,S2,June,fff05995_agbm.tif
189076,252401,fff05995_S2_10.tif,fff05995,S2,July,fff05995_agbm.tif


In [18]:
train = train[train.satellite == 'S2']

In [22]:
grouped = train.groupby('chip_id').agg(list)[['filename', 'month', 'corresponding_agbm']]
grouped

,filename,month,corresponding_agbm
chip_id,,,
0003d2eb,"[0003d2eb_S2_00.tif, 0003d2eb_S2_01.tif, 0003d...","[September, October, February, March, April, M...","[0003d2eb_agbm.tif, 0003d2eb_agbm.tif, 0003d2e..."
000aa810,"[000aa810_S2_06.tif, 000aa810_S2_07.tif, 000aa...","[March, April, May, June, July, August]","[000aa810_agbm.tif, 000aa810_agbm.tif, 000aa81..."
000d7e33,"[000d7e33_S2_06.tif, 000d7e33_S2_07.tif, 000d7...","[March, April, May, June, July, August]","[000d7e33_agbm.tif, 000d7e33_agbm.tif, 000d7e3..."
00184691,"[00184691_S2_06.tif, 00184691_S2_07.tif, 00184...","[March, April, May, June, July, August]","[00184691_agbm.tif, 00184691_agbm.tif, 0018469..."
001b0634,"[001b0634_S2_00.tif, 001b0634_S2_01.tif, 001b0...","[September, October, November, January, Februa...","[001b0634_agbm.tif, 001b0634_agbm.tif, 001b063..."
...,...,...,...
ffc563f4,"[ffc563f4_S2_00.tif, ffc563f4_S2_01.tif, ffc56...","[September, October, November, January, Februa...","[ffc563f4_agbm.tif, ffc563f4_agbm.tif, ffc563f..."
ffc730b9,"[ffc730b9_S2_00.tif, ffc730b9_S2_01.tif, ffc73...","[September, October, November, January, Februa...","[ffc730b9_agbm.tif, ffc730b9_agbm.tif, ffc730b..."
ffc7d4f2,"[ffc7d4f2_S2_00.tif, ffc7d4f2_S2_01.tif, ffc7d...","[September, October, November, January, Februa...","[ffc7d4f2_agbm.tif, ffc7d4f2_agbm.tif, ffc7d4f..."


In [23]:
labels = []
for chip_id, chip in grouped.iterrows():
	assert len(set(chip.corresponding_agbm)) == 1
	labels.append(chip['corresponding_agbm'][0])
grouped['corresponding_agbm'] = labels
grouped

,filename,month,corresponding_agbm
chip_id,,,
0003d2eb,"[0003d2eb_S2_00.tif, 0003d2eb_S2_01.tif, 0003d...","[September, October, February, March, April, M...",0003d2eb_agbm.tif
000aa810,"[000aa810_S2_06.tif, 000aa810_S2_07.tif, 000aa...","[March, April, May, June, July, August]",000aa810_agbm.tif
000d7e33,"[000d7e33_S2_06.tif, 000d7e33_S2_07.tif, 000d7...","[March, April, May, June, July, August]",000d7e33_agbm.tif
00184691,"[00184691_S2_06.tif, 00184691_S2_07.tif, 00184...","[March, April, May, June, July, August]",00184691_agbm.tif
001b0634,"[001b0634_S2_00.tif, 001b0634_S2_01.tif, 001b0...","[September, October, November, January, Februa...",001b0634_agbm.tif
...,...,...,...
ffc563f4,"[ffc563f4_S2_00.tif, ffc563f4_S2_01.tif, ffc56...","[September, October, November, January, Februa...",ffc563f4_agbm.tif
ffc730b9,"[ffc730b9_S2_00.tif, ffc730b9_S2_01.tif, ffc73...","[September, October, November, January, Februa...",ffc730b9_agbm.tif
ffc7d4f2,"[ffc7d4f2_S2_00.tif, ffc7d4f2_S2_01.tif, ffc7d...","[September, October, November, January, Februa...",ffc7d4f2_agbm.tif


In [24]:
max_len, max_len_chip_id = 0, None
for chip_id, group in grouped.iterrows():
	if len(group.month) > max_len:
		max_len = len(group.month)
		max_len_chip_id = chip_id

max_len, max_len_chip_id

(12, '0060c0a5')

In [25]:
months = grouped.month.loc[max_len_chip_id]
months

['September',
 'October',
 'November',
 'December',
 'January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August']

In [40]:
filenames = []
for chip_id, group in grouped.iterrows():
	filenames.append([None]*len(months))
	for i,m in enumerate(months):
		if m in group.month:
			filenames[-1][i] = group.filename[group.month.index(m)]

grouped2 = pd.DataFrame({'filename': filenames, 'corresponding_agbm': labels[1]})
grouped2

,filename,corresponding_agbm
0,"[0003d2eb_S2_00.tif, 0003d2eb_S2_01.tif, None,...",000aa810_agbm.tif
1,"[None, None, None, None, None, None, 000aa810_...",000aa810_agbm.tif
2,"[None, None, None, None, None, None, 000d7e33_...",000aa810_agbm.tif
3,"[None, None, None, None, None, None, 00184691_...",000aa810_agbm.tif
4,"[001b0634_S2_00.tif, 001b0634_S2_01.tif, 001b0...",000aa810_agbm.tif
...,...,...
8684,"[ffc563f4_S2_00.tif, ffc563f4_S2_01.tif, ffc56...",000aa810_agbm.tif
8685,"[ffc730b9_S2_00.tif, ffc730b9_S2_01.tif, ffc73...",000aa810_agbm.tif
8686,"[ffc7d4f2_S2_00.tif, ffc7d4f2_S2_01.tif, ffc7d...",000aa810_agbm.tif
8687,"[ffd8db7a_S2_00.tif, ffd8db7a_S2_01.tif, ffd8d...",000aa810_agbm.tif
